In [39]:
!python --version

Python 3.11.5


In [18]:
import pandas as pd
import re
from collections import defaultdict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pymorphy3
from rapidfuzz import process, fuzz

In [14]:
def clean_and_reduce_skills(data, column_name='key_skills', threshold=80):
    """
    Очищает, нормализует и сокращает количество уникальных навыков в указанном столбце.
    
    Параметры:
        data (pd.DataFrame): Исходный DataFrame.
        column_name (str): Название столбца с навыками.
        threshold (int): Порог для объединения похожих навыков (0-100).
    
    Возвращает:
        list: Отсортированный и уменьшенный список уникальных навыков.
    """
    # Шаг 1: Удаляем пропущенные значения и разделяем навыки
    all_skills = data[column_name].dropna().str.split(',').sum()
    
    # Шаг 2: Приводим к нижнему регистру и удаляем лишние пробелы
    cleaned_skills = {skill.strip().lower() for skill in all_skills if skill.strip()}
    
    # Шаг 3: Объединяем похожие навыки
    reduced_skills = []
    while cleaned_skills:
        skill = cleaned_skills.pop()
        similar_skills = process.extract(skill, cleaned_skills, scorer=fuzz.ratio, score_cutoff=threshold)
        reduced_skills.append(skill)
        # Удаляем все найденные похожие навыки
        for match, _, _ in similar_skills:
            cleaned_skills.discard(match)
    
    # Шаг 4: Сортируем результат
    return sorted(reduced_skills)


In [2]:
def normalize_russian_words(words):
    """
    Приводит русский текст к нормальной форме.
    """
    morph = pymorphy3.MorphAnalyzer()
    return [morph.parse(word)[0].normal_form for word in words]

def add_features_to_dataframe(data, features, skills_column='key_skills'):
    """
    Добавляет столбцы для признаков и отмечает их наличие в ключевых навыках.

    Параметры:
        data (pd.DataFrame): Исходный DataFrame.
        features (list): Список признаков.
        skills_column (str): Название столбца с ключевыми навыками.

    Возвращает:
        pd.DataFrame: DataFrame с добавленными столбцами признаков.
    """
    # Удаляем дубликаты из списка признаков
    features = list(set(features))
    
    # Отделяем русские слова от остальных
    russian_words = [feature for feature in features if re.search(r'[а-яА-Я]', feature)]
    other_words = [feature for feature in features if feature not in russian_words]
    
    # Нормализуем русские слова
    normalized_russian_words = normalize_russian_words(russian_words)
    
    # Объединяем нормализованные русские слова с другими
    final_features = normalized_russian_words + other_words

    # Добавляем столбцы с признаками
    for feature in final_features:
        data[feature] = 0  # Инициализация столбцов с нулями
    
    morph = pymorphy3.MorphAnalyzer()

    # Проверяем навыки в key_skills построчно
    for index, row in data.iterrows():
        key_skills = str(row[skills_column]).lower().replace(' ', '_').replace('-', '_').split(', ')
        for feature in final_features:
            if feature in key_skills:
                f = morph.parse(feature)[0].normal_form
                print(f)
                data.loc[index, f] = 1  # Помечаем 1, если навык найден

    return data

In [3]:
with open("../data/client_dataset.json", "r", encoding="utf-8") as file:
    data = json.load(file)
    
data = pd.DataFrame(data)

In [19]:
clean_and_reduce_skills(data)

['!с: erp',
 '#c',
 '(elk',
 '(java 8 (java-core',
 '(jpa',
 '(mockups',
 '(n)hibernate',
 '(opensearch managed)',
 '(vipnet',
 '(web flux',
 '(линейная/логистическая',
 ')',
 '* client server ios application * ble communication with central peripheral architecture * cardfli',
 '* быстрый и способный внедрять новые передовые технологии',
 '* глубоком знании всех компонентов android: фрагментов',
 '* ключевой разработчик android',
 '* мониторинге стабильности приложений',
 '* обзор кода',
 '* охват тестированием',
 '* последовательность в выполнении работы в срок',
 '* производительности и настройке приложений (jvm / android)',
 '* разработка приложений',
 '* разработке нативного android',
 '* разработчик android java/kotlin',
 '* рефакторинг и редизайн существующих функций',
 '* страсть к технологиям и разработке программного обеспечения',
 '*nix',
 '*nix services: apache',
 '+c4',
 '- always understand other people or quickly clarify if there is a misunderstanding between my vision an

In [20]:
processed_data = add_features_to_dataframe(data, features)
processed_data['c++'].sum()

ответственность
ответственность
ответственность
ответственность
обучаемость
go
ответственность
обучаемость
java
ответственность
ответственность
обучаемость
ответственность
ответственность
ответственность
ответственность
ответственность
ответственность
обучаемость
ответственность
java
ответственность
ответственность
ответственность
обучаемость
ответственность
java
обучаемость
обучаемость
обучаемость
обучаемость
ответственность
обучаемость
обучаемость
обучаемость
ответственность
ответственность
ответственность
ответственность
обучаемость
обучаемость
ответственность
ответственность
обучаемость
обучаемость
обучаемость
ответственность
обучаемость
обучаемость
обучаемость
ответственность
ответственность
ответственность
обучаемость
ответственность
ответственность
обучаемость
обучаемость
обучаемость
java
java


0